In [5]:
# This should be built on a fully commissioned ESP32
# But there's problems with getting both i2c devices 
# working from an ESP32

# So we have an MQTT version which you can get the data using
# mosquitto_sub -h 192.168.0.159 -v -t "walkingpole/#"

In [267]:
%serialconnect

Connecting to Serial /dev/ttyUSB0 baud=115200 
Ready.


In [99]:
# the location of the i2c pins on the firebeetle
%sendtofile i2cmodule.py
import machine
#i2c = machine.I2C(scl=machine.Pin(22), sda=machine.Pin(21))
i2c = machine.I2C(scl=machine.Pin(4), sda=machine.Pin(5), freq=100000)  #D2,D1 NodeMCU


Sent 3 lines (163 bytes) to i2cmodule.py.


In [161]:
%sendtofile --source ../essential_sensor_code/VL53L0X_funcs.py
%sendtofile --source ../essential_sensor_code/BNO055_funcs.py

Sent 46 lines (1653 bytes) to BNO055_funcs.py.


In [142]:
from VL53L0X_funcs import VL53L0Xinit, VL53L0Xdist

In [215]:
import time
VL53L0Xinit()
for i in range(10):
    print(VL53L0Xdist())
    time.sleep(1)



***Connection broken [Input/output error]
You may need to reconnect

***OSError [Input/output error]



In [158]:
from i2cmodule import i2c
print(i2c.scan())

[40, 41]


In [160]:
k = i2c.readfrom_mem(0x28, 0x00, 6)
print("BNO055 sensor SW_REV_ID: %s.%s" %(hex(k[4]), hex(k[5])))


BNO055 sensor SW_REV_ID: 0x11.0x3


In [162]:
from BNO055_funcs import BNO055init, BNO055calibstat, BNO055quat, BNO055pitchrollorient


In [178]:
#BNO055init()
print(BNO055quat())

(15342, 24, -366, -5739)


In [63]:
print(VL53L0Xdist())


Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
NameError: name 'VL53L0Xdist' is not defined


In [274]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to Serial /dev/ttyUSB0 baud=115200 
Ready.


In [275]:
%sendtofile main.py

#ssid, ssidpassword = b'DoESLiverpool', b'decafbad00'
#mqttbroker = "192.168.0.159"   # mqtt.local
ssid = None

from VL53L0X_funcs import VL53L0Xinit, VL53L0Xdist
from BNO055_funcs import BNO055init, BNO055calibstat, BNO055quat

from machine import Pin
import time, network
from umqtt.robust import MQTTClient

pled = Pin(2, Pin.OUT)

if ssid:
    staif = network.WLAN(network.STA_IF)
    staif.active(True)
    staif.connect(ssid, ssidpassword)
    while not staif.isconnected():
        pled.value(1-pled.value())
        time.sleep(0.2)
else:
    staif = network.WLAN(network.AP_IF)
    staif.active(True)
    
VL53L0Xinit()
BNO055init()

for i in range(1000):
    pled.value(1-pled.value())
    if not ssid:
        mqttbroker = "192.168.4.%d"%((i%4)+2)
    try:
        print("mqttbroker", mqttbroker)
        c = MQTTClient(b"walkingpole", mqttbroker)
        c.connect()
        break
    except OSError as e:
        print(e)
c.publish(b"walkingpole/ip", staif.ifconfig()[0].encode())

t0 = time.ticks_ms()
while True:
    pled.value(1)
    time.sleep_ms(5)
    t1 = time.ticks_ms()
    time.sleep_ms(max(5, 50-(t1-t0)))
    t1 = time.ticks_ms()
    pled.value(0)
    msg = b"{} {} {} {} {} {}".format(t1-t0, VL53L0Xdist(), *BNO055quat())
    print(msg)
    c.publish(b"walkingpole/quatd", msg)
    t0 = t1



Sent 53 lines (1316 bytes) to main.py.


In [205]:
x = [1,2]
def f(a,b,c):
    print(c)
f(9, *x)

2


In [223]:
help(network)

object <module 'network'> is of type module
  __name__ -- network
  WLAN -- <function>
  phy_mode -- <function>
  STA_IF -- 0
  AP_IF -- 1
  STAT_IDLE -- 0
  STAT_CONNECTING -- 1
  STAT_WRONG_PASSWORD -- 2
  STAT_NO_AP_FOUND -- 3
  STAT_CONNECT_FAIL -- 4
  STAT_GOT_IP -- 5
  MODE_11B -- 1
  MODE_11G -- 2
  MODE_11N -- 3
  AUTH_OPEN -- 0
  AUTH_WEP -- 1
  AUTH_WPA_PSK -- 2
  AUTH_WPA2_PSK -- 3
  AUTH_WPA_WPA2_PSK -- 4


In [224]:
import network
staif = network.WLAN(network.AP_IF)
staif.active(True)


In [242]:
help(staif)

object <WLAN> is of type WLAN
  active -- <function>
  connect -- <function>
  disconnect -- <function>
  status -- <function>
  scan -- <function>
  isconnected -- <function>
  config -- <function>
  ifconfig -- <function>


In [245]:
print(staif.config("essid"))
print(staif.status("stations"))


MicroPython-ac74e4


Traceback (most recent call last):
  File "<stdin>", line 2, in <module>
TypeError: function takes 1 positional arguments but 2 were given


In [263]:
client = MQTTClient(b"walkingpole", '192.168.4.4')
client.connect()
client.publish(b"walkingpole/ip", staif.ifconfig()[0].encode())

In [265]:
client.publish(b"walkingpole/duung", b"dong")


In [268]:
%disconnect

attempt to exit paste mode
[\r\x03\x02] b'\r\nMicroPython v1.9.3-8-g63826ac5c on 2017-11-01; ESP module with ESP8266\r\nType "help()" for more information.\r\n>>> '
Closing serial Serial<id=0x7f237e71b198, open=True>(port='/dev/ttyUSB0', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)
